## Подход 1. Градиентный бустинг "в лоб"

In [15]:
import pandas as pd
import numpy as np
from src.utils import out
from scipy import stats
from sklearn.preprocessing import StandardScaler
import datetime
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import numpy as np
from joblib import dump, load
import pandas as pd

class CList:
    def __init__(self, start, stop, step):
        self.start = start
        self.stop = stop
        self.step = step
    def __iter__(self):
        self.c = self.start
        return self
    def __next__(self):
        if self.c <= self.stop:
            x = self.c
            self.c += self.step
            return x
        else:
            raise StopIteration

1.

Считать таблицу с признаками из файла.
Уменьшить колличество векторов в 3 раза для ускорения кросс валидации.

In [37]:
data = pd.read_csv('./features.csv', index_col='match_id')
data = data.sample(n=round(len(features.index) / 3))

In [38]:
data.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
1468,1431032282,1,25,5,1807,1106,12,0,1,7,...,3,2,1,-41.0,1328,1,1975,1796,63,59
39325,1443324979,1,50,3,1078,849,10,0,0,9,...,5,3,0,-46.0,2738,0,1792,1974,50,63
55206,1445954799,7,86,3,894,598,1,0,0,9,...,2,3,1,25.0,2739,1,1958,4,63,3
43870,1444068935,7,48,4,1307,1624,24,0,0,8,...,4,2,0,6.0,1735,1,1983,260,63,51
105691,1449725601,1,20,4,1467,1227,17,0,2,15,...,4,2,0,-37.0,2230,0,1824,1974,60,63


Удалить признаки связанные с итогами матча

In [39]:
columns_to_drop = ['duration', 'tower_status_radiant', 'tower_status_dire',
                       'barracks_status_radiant',
                       'barracks_status_dire']
data = data.drop(columns_to_drop, axis=1)

2.

Проверить выборку на наличие пропусков

In [40]:
values_full = data.count()
rows_quantity = data.shape[0]
columns_with_empty = []
for column, length in values_full.iteritems():
    if rows_quantity - length > 0:
        columns_with_empty.append(column)
columns_with_empty

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

Данные колонки имеют пропущенные значения, по скольку не всегда описуемые данными колонками события происходят в первые 5 минут игры.

3.

Заменить пропуски на 0

In [41]:
value = 0
data[columns_with_empty] = data[columns_with_empty].fillna(value=value, inplace=False)

4.
Отделить колонку с целевой переменной.
X - фичи
y - целевая переменная, колонка 'radiant_win'

In [42]:
y = data[["radiant_win"]].values.ravel()
X = data.drop(["radiant_win"], axis=1).values

5.
Фиксировать генератор разбиений по 5 блокам

In [43]:
kf = KFold(n_splits=5, shuffle=True)

Оценить качество градиентного бустинга при разном колличестве деревьев

In [44]:
n_estimators = 40
gb_trees_step = 10
qualities = []
for i in range(gb_trees_step, n_estimators, gb_trees_step):
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n_estimators)
    qualities_n_estimators = []
    for train_index, test_index in kf.split(X):
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf = clf.fit(x_train, y_train)
        predictions = clf.predict_proba(x_test)[:, 1]
        qualities_n_estimators.append(roc_auc_score(y_test, predictions))
    mean_quality = np.mean(qualities_n_estimators)
    qualities.append(mean_quality)
    print("Trees quantity: " + str(i))
    print('Time:', datetime.datetime.now() - start_time)
    print("Quality AUC-ROC: " + str(mean_quality))


Trees quantity: 10
Time: 0:00:30.832353
Quality AUC-ROC: 0.6883560001959085
Trees quantity: 20
Time: 0:00:31.470342
Quality AUC-ROC: 0.6888734059306336
Trees quantity: 30
Time: 0:00:30.793391
Quality AUC-ROC: 0.6889569129140193


Длительность кросс-валидации для градиентного бустинга с 30 деревьями: 0:00:30.793391
Качество по AUC - ROC: 0.6889569129140193

В градиентном бустинге имеет смысл использовать больше 30 деревьев, так как это даёт прирост качества

Для ускорения обучения можно:
1. Уменьшение максимальной глубины деревьев (параметр max_depth)
2. Использование случайно выбранной подбвыборки для обучения и кросс-валидации
3. Удаление столбцов, feature_importance которых равняется 0 при текущем колличестве деревьев.

## Подход 2. Логистическая регрессия

In [119]:
data = pd.read_csv('./features.csv', index_col='match_id')
columns_to_drop = ['duration', 'tower_status_radiant', 'tower_status_dire',
                       'barracks_status_radiant',
                       'barracks_status_dire']
data = data.drop(columns_to_drop, axis=1)
data[columns_with_empty] = data[columns_with_empty].fillna(value=0, inplace=False)
x_data = data.drop(["radiant_win"], axis=1)
y = data[["radiant_win"]].values.ravel()
X = x_data.values

Необходимо масштабировать данные, так как логистическая регрессия чувствительна к выбросам

In [120]:
def scale_data(X_train):
    scaler = StandardScaler()
    scaler.fit(X_train)
    StandardScaler(copy=True, with_mean=True, with_std=True)
    X_train = scaler.transform(X_train)
    return X_train

X = scale_data(X)
X

array([[-2.54436416,  1.54068827, -1.24422828, ..., -0.55115386,
         1.84600409, -1.12149424],
       [-2.54045236, -0.92779756, -0.29225805, ...,  0.67817009,
         0.43778816,  0.04394713],
       [-2.53923104,  1.54068827, -0.5686365 , ...,  0.67817009,
         0.43778816,  0.49028637],
       ...,
       [ 1.09874571, -0.57515673,  1.42743012, ...,  0.67817009,
         0.43778816, -0.20401912],
       [ 1.09895204, -0.57515673,  1.48884755, ...,  0.67817009,
         0.43778816, -0.87352799],
       [ 1.1026479 ,  1.54068827, -0.04658831, ..., -0.55115386,
        -0.97042777, -0.79913812]])

Код для подборки параметра С и получения лучшего качества вынесен в отдельные функции для удобства.

In [121]:
def process_lr(kf, X, y, c_list_iterator):
    qualities = []
    for i, c in enumerate(c_list_iterator):
        start_time = datetime.datetime.now()
        clf = LogisticRegression(penalty='l2', C=c, solver="lbfgs", max_iter=200)
        qualities_c = []
        for train_index, test_index in kf.split(X):
            x_train, x_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            clf = clf.fit(x_train, y_train)
            predictions = clf.predict_proba(x_test)[:, 1]
            qualities_c.append(roc_auc_score(y_test, predictions))

        mean_quality = round(np.mean(qualities_c), 5)
        qualities.append(mean_quality)

        # print("C: " + str(c))
        print('Time:', datetime.datetime.now() - start_time)
        # print("Quality AUC-ROC: " + str(mean_quality))
        # print("-")

    return qualities

def get_best_quality_c(qualities, start, step):
    max_q = max(qualities)
    best_c = (qualities.index(max_q) * step) + start
    return max_q, best_c

In [122]:
kf = KFold(n_splits=5, shuffle=True)
start, stop, step = 1, 4, 0.2
c_list_iterator = iter(CList(start, stop, step))

qualities = process_lr(kf, X, y, c_list_iterator)
max_q, best_c = get_best_quality_c(qualities, start, step)

print("Max quality: ", max_q)
print("Max quality C: ", best_c)

Time: 0:00:05.510391
Time: 0:00:05.384582
Time: 0:00:05.461654
Time: 0:00:04.666510
Time: 0:00:05.536308
Time: 0:00:05.862188
Time: 0:00:05.193675
Time: 0:00:05.194138
Time: 0:00:05.033508
Time: 0:00:05.281753
Time: 0:00:05.155018
Time: 0:00:05.251207
Time: 0:00:05.195515
Time: 0:00:05.310268
Time: 0:00:05.612464
Max quality:  0.71658
Max quality C:  3.2


Таким образом:
- Качество у логистической регрессии над всеми исходными признаками: ~0.71658
- Выше качества градиентного бустинга на ~2.5%. Т.е логистическая лучше находит зависимости в текущих данных.
- Логистическая регрессия работает намного быстрее градиентного бустинга.

Попробовать удалить категориальные признаки, которые присутствуют в выборке и используются как числовые и провести кросс валидацию с подбором параметра на новой выборке:

In [123]:
# Удалить колонки с категориальными признаками
x_data_cat_dropped = x_data.drop(['lobby_type'], axis=1)
for i in range(1, 6):
    x_data_cat_dropped = x_data_cat_dropped.drop(["r" + str(i) + "_hero"], axis=1)
    x_data_cat_dropped = x_data_cat_dropped.drop(["d" + str(i) + "_hero"], axis=1)

In [124]:
# Масштабировать данные, провести кросс валидицаю с подбором параметра

X_cat_dropped = scale_data(x_data_cat_dropped.values)

qualities = process_lr(kf, X_cat_dropped, y, c_list_iterator)
max_q, best_c = get_best_quality_c(qualities, start, step)

print("Max quality: ", max_q)
print("Max quality C: ", best_c)

Time: 0:00:05.271552
Time: 0:00:05.467514
Time: 0:00:05.734954
Time: 0:00:05.558326
Time: 0:00:05.468697
Time: 0:00:05.101692
Time: 0:00:05.345313
Time: 0:00:05.170439
Time: 0:00:05.446581
Time: 0:00:05.075512
Time: 0:00:05.237861
Time: 0:00:05.009767
Time: 0:00:05.162379
Time: 0:00:05.523853
Time: 0:00:05.310738
Max quality:  0.71661
Max quality C:  1.4


Как можно видеть, качество не сильно изменилось (0.71658 C = 3.2 и 0.71661 C = 1.4). Это можно обосновать тем, что использование категориальных признаков в форме числовых не несёт в себе какой-либо важной информации для определения победителя (бессмысленно сравнивать id героев)

Колличество уникальных идентификаторов героев:

In [125]:
hero_columns = []
for i in range(1, 6):
    hero_columns.append("r" + str(i) + "_hero")
    hero_columns.append("d" + str(i) + "_hero")
unic_ids = pd.unique(data[hero_columns].values.ravel())
heroes_quantity = len(unic_ids)
print("Unic heroes:", sorted(unic_ids))
print("Unic heroes id quantity:", heroes_quantity)

Unic heroes: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 109, 110, 112]
Unic heroes id quantity: 108


Можно видеть, что всего героев в доте 112, однако в выборке присутствуют не все.

Воспользуемся подходом "мешок слов" для кодирования информации о героях. Сформировать N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire.

In [126]:
def process_categorial(heroes_quantity, x_data, X):
    X_pick = np.zeros((x_data.shape[0], heroes_quantity + 4))

    for i, match_id in enumerate(x_data.index):
        for p in range(5):
            X_pick[i, x_data.loc[match_id, 'r%d_hero' % (p + 1)] - 1] = 1
            X_pick[i, x_data.loc[match_id, 'd%d_hero' % (p + 1)] - 1] = -1
    return np.hstack((X, X_pick))

X = process_categorial(heroes_quantity, x_data, X)

Провести кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации:


In [127]:
qualities = process_lr(kf, X, y, c_list_iterator)
max_q, best_c = get_best_quality_c(qualities, start, step)

print("Max quality: ", max_q)
print("Max quality C: ", best_c)

Time: 0:00:22.198965
Time: 0:00:23.716637
Time: 0:00:24.036488
Time: 0:00:23.381655
Time: 0:00:24.070064
Time: 0:00:22.929197
Time: 0:00:23.858598
Time: 0:00:24.831898
Time: 0:00:24.246632
Time: 0:00:24.221710
Time: 0:00:22.411355
Time: 0:00:22.479830
Time: 0:00:22.382260
Time: 0:00:22.026595
Time: 0:00:22.823755
Max quality:  0.75204
Max quality C:  1.4


Качество: 0.75204 при С = 1.4. После использование мешка слов качество заметно улучшилось. Это связано с тем, что комманда имеет больший шанс выйграть, если в ней присутствуют определенные герои.

Обучим и сохраним итоговую модель:

In [128]:
lr = LogisticRegression(penalty='l2', C=best_c, solver="lbfgs", max_iter=200)
for train_index, test_index in kf.split(X):
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        lr = lr.fit(x_train, y_train)
dump(lr, "lr.joblib")


['lr.joblib']

## Проверка финальной модели

In [135]:
def prepare_test_data_lr():
    # Считать набор данных для теста
    data = pd.read_csv('./features_test.csv', index_col='match_id')
    # Заполнить пропущенные значения
    data[columns_with_empty] = data[columns_with_empty].fillna(value=0, inplace=False)
    # Масштабировать данные
    X_scaled = scale_data(data.values)
    # Подготовить категориальные данные
    X = process_categorial(108, data, X_scaled)
    return X, data

def test_lr(X, data):
    # Загрузить сохраненную модель логистической регрессии
    clf = load("lr.joblib")
    predictions_df = pd.DataFrame(clf.predict_proba(X))
    predictions_df.columns = ['dire_win', 'radiant_win']
    predictions_df.index = data.index
    print(predictions_df.head())
    print("Mean:\n", predictions_df.mean())
    print("Max prob. radiant win: ", max(predictions_df['radiant_win']))  # 0.9966559346546404
    print("Min prob. radiant win: ", min(predictions_df['radiant_win']))  # 0.00956029948527895
    result = predictions_df.drop(['dire_win'], axis=1)
    result.to_csv('./result.csv')

In [136]:
X, data = prepare_test_data_lr()
test_lr(X, data)

          dire_win  radiant_win
match_id                       
6         0.175828     0.824172
7         0.244883     0.755117
10        0.816023     0.183977
13        0.161980     0.838020
16        0.776659     0.223341
Mean:
 dire_win       0.481833
radiant_win    0.518167
dtype: float64
Max prob. radiant win:  0.9966875459601456
Min prob. radiant win:  0.008568222184926852


Максимальная вероятность победить у radiant: 0.9966875459601456;
Минимальная вероятность победить у radiant: 0.008568222184926852;
Таким образом предсказания находятся на отрезке [0:1].
Также можно сделать вывод, что в среднем вероятность победы у radiant немного выше, чем у dire.